In [73]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

In [74]:
df = pd.read_csv('image_labels.csv')
df.head()

,image,label,left,width,top,height
0,57503_000116_Endzone_frame443.jpg,Helmet,1099,16,456,15
1,57503_000116_Endzone_frame443.jpg,Helmet,1117,15,478,16
2,57503_000116_Endzone_frame443.jpg,Helmet,828,16,511,15
3,57503_000116_Endzone_frame443.jpg,Helmet,746,16,519,16
4,57503_000116_Endzone_frame443.jpg,Helmet,678,17,554,17


In [75]:
df['x_center'] = df['left'] + df['width'] / 2
df['y_center'] = df['top'] + df['height'] / 2

df['x_center_scaled'] = df['x_center'] / 1280
df['y_center_scaled'] = df['y_center'] / 720

df['width_scaled'] = df['width'] / 1280
df['height_scaled'] = df['height'] / 720

print(df.shape)
df.head()

(193736, 12)


,image,label,left,width,top,height,x_center,y_center,x_center_scaled,y_center_scaled,width_scaled,height_scaled
0,57503_000116_Endzone_frame443.jpg,Helmet,1099,16,456,15,1107.0,463.5,0.864844,0.643750,0.012500,0.020833
1,57503_000116_Endzone_frame443.jpg,Helmet,1117,15,478,16,1124.5,486.0,0.878516,0.675000,0.011719,0.022222
2,57503_000116_Endzone_frame443.jpg,Helmet,828,16,511,15,836.0,518.5,0.653125,0.720139,0.012500,0.020833
3,57503_000116_Endzone_frame443.jpg,Helmet,746,16,519,16,754.0,527.0,0.589063,0.731944,0.012500,0.022222
4,57503_000116_Endzone_frame443.jpg,Helmet,678,17,554,17,686.5,562.5,0.536328,0.781250,0.013281,0.023611


In [76]:
if not os.path.exists('labels'):
    os.mkdir('labels')

In [77]:
%%time

filenames = []

for img in tqdm(sorted(df.image.unique()), total=len(df.image.unique())):
    filenames.append('data/custom/images/' + img)
    
    temp = df[df.image == img]
    annotations = []
    for _,row in temp.iterrows():
        annotations.append([0, row['x_center_scaled'], row['y_center_scaled'], row['width_scaled'], row['height_scaled']])

    np_annotations = np.array(annotations)
    np.savetxt('labels/' + str(img)[:-4] + '.txt', np_annotations, fmt='%i %1.4f %1.4f %1.4f %1.4f')


CPU times: user 2min 32s, sys: 7.39 s, total: 2min 40s
Wall time: 3min 6s


In [78]:
train_idx = int(len(filenames) * 0.8)
train, test = filenames[:train_idx], filenames[train_idx:]
np_train = np.array(train)
np_test = np.array(test)
np.savetxt('train.txt', np_train, fmt='%s')
np.savetxt('valid.txt', np_test, fmt='%s')